In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os

# print("sys.path", sys.path)

import matplotlib
import matplotlib.pyplot as plt

# import seaborn as sns
import json
import warnings
from pathlib import Path
import random
import base64
from io import BytesIO
import cv2
# import vision_pipeline.obb
# import imagesize
# from scipy import ndimage
import natsort
from PIL import Image as PILImage
# from PIL import ImageDraw, ImageFilter
import numpy as np
from tqdm import tqdm
# from shapely.geometry import Polygon
# from rich import print
# from types import SimpleNamespace
import pickle
import imutils

# ros package
from context_action_framework.types import Detection, Label, Module, Camera, detections_to_ros, detections_to_py
from sensor_msgs.msg import Image, CameraInfo # CameraInfo needed for pickle

from context_action_framework.srv import VisionDetection, VisionDetectionResponse, VisionDetectionRequest, ProcessImg, ProcessImgResponse

from context_action_framework.graph_relations import GraphRelations

from cv_bridge import CvBridge
import rospy


rospy.init_node("test_node")


In [2]:
img_path = os.path.expanduser("~/datasets2/reconcycle/2023-08-01_basler_hca_backs/0001.jpg")
# img_path = os.path.expanduser("~/datasets2/reconcycle/2023-12-04_hcas_fire_alarms_sorted/firealarm_back_01/00_template_0039.jpg")
# img_path = os.path.expanduser("~/datasets2/reconcycle/2023-12-04_hcas_fire_alarms_sorted/firealarm_back_03.1/0182.jpg")


img = cv2.imread(img_path)
img = imutils.resize(img, width=1450, height=1450)

# display(PILImage.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))

In [3]:
def call_process_img(img):
    timeout = 3 # 2 second timeout
    rospy.wait_for_service('vision/basler/process_img', timeout)

    imgmsg = CvBridge().cv2_to_imgmsg(img, encoding="bgr8")
    try:
        process_img = rospy.ServiceProxy('vision/basler/process_img', ProcessImg)
        response = process_img(imgmsg)
        detections = detections_to_py(response.detections)
        labelled_img = CvBridge().imgmsg_to_cv2(response.labelled_image, desired_encoding='passthrough')
        cropped_img = CvBridge().imgmsg_to_cv2(response.cropped_image, desired_encoding='passthrough')
        
        return response.success, detections, labelled_img, cropped_img
        # return response
    except rospy.ServiceException as e:
        print("Service call failed: %s"%e)



In [5]:
success, detections, labelled_img, cropped_img = call_process_img(img)

display(PILImage.fromarray(cv2.cvtColor(labelled_img, cv2.COLOR_BGR2RGB)))
display(PILImage.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)))

print("detections", detections)

MarkupError: closing tag '[/vision/basler/process_img]' at position 29 doesn't match any open tag

In [ ]:

# given img, return:
# - positions of objects (incl. gap)
# - relations of objects
# - precise obj recognition


success, detections, labelled_img, cropped_img = call_process_img(img)


graph_relations = GraphRelations(detections)

A_relations = graph_relations.to_text()


label_precise = None
positions_list = []
for detection in detections:
    if detection.label_precise != "":
        label_precise = detection.label_precise
    positions_list.append(f"{detection.label.name} at ({detection.center_px[0]}, {detection.center_px[1]})")

A_positions = ", ".join(positions_list) + ". "

print("A_relations", A_relations)
print("A_positions", A_positions)
print("label_precise", label_precise)



A_relations Device 1: internals in hca_back, battery in hca_back, battery next to internals.

A_positions hca_back at (982.0, 1158.0), internals at (985.0, 1162.0), battery at (1060.0, 1203.0).

label_precise hca_back_03

In [ ]:


# def call_get_detection():
#     timeout = 3 # 2 second timeout
#     service_path = 'vision/basler/get_detection'
#     rospy.wait_for_service(service_path, timeout)

#     # imgmsg = CvBridge().cv2_to_imgmsg(img, encoding="bgr8")
#     try:
#         get_detection = rospy.ServiceProxy(service_path, VisionDetection)
#         response = get_detection(camera=Camera.basler.value, gap_detection=False)
#         return response
#     except rospy.ServiceException as e:
#         print("Service call failed: %s"%e)

# response = call_get_detection()
# print("response", response)